In [1]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00


In [ ]:
# -------------------------- YOLOv8 Detection --------------------------


import os
import pandas as pd
from ultralytics import YOLO

# Base folder with images
base_dir = "/content/drive/MyDrive"
folders = ["SDXL_1.0", "SD2", "Flux-Dev"]

# Load YOLOv8 model
yolo_model = YOLO("yolov8x.pt")

for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    results_list = []

    for img_file in os.listdir(folder_path):
        if img_file.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder_path, img_file)

            try:
                # Run detection
                result = yolo_model(img_path)[0]
                boxes = result.boxes.xyxy.cpu().numpy()
                scores = result.boxes.conf.cpu().numpy()
                labels = result.boxes.cls.cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    results_list.append({
                        "image_name": img_file,
                        "bbox": box.tolist(),
                        "score": float(score),
                        "label": int(label)
                    })
            except Exception as e:
                print(f" Error processing {img_file}: {e}")

    # Save CSV per folder
    csv_path = os.path.join(base_dir, f"{folder}_yolov8_detections.csv")
    pd.DataFrame(results_list).to_csv(csv_path, index=False)
    print(f" Saved YOLOv8 detections for {folder} to {csv_path}")
